In [279]:
import pandas as pd
import sklearn as skt
import re
from datetime import datetime, timezone
import matplotlib.pyplot as plt

In [280]:
# Criar arquivo CSV
def baseArchive():
    basePath = os.path.join(os.getcwd(), 'db', 'FakeReal.csv')
    os.makedirs(os.path.dirname(basePath), exist_ok=True)
    if not os.path.exists(basePath):
        with open(basePath, 'w', encoding='utf-8') as baseArchive:
            baseArchive.write('titulo,URL\n')  # Cabeçalho compatível com os dados
    print(f"Arquivo base criado ou já existente em: {basePath}")
    return basePath

In [281]:
def saveData(data, filename=None):
    if filename is None:
        filename = baseArchive()

    df = pd.DataFrame(data)

    try:
        # Caso o arquivo já exista
        if os.path.exists(filename):
            existing_df = pd.read_csv(filename)
            
            # Combina os novos dados com os existentes, sem remover duplicatas
            combined_df = pd.concat([existing_df, df], ignore_index=True)
            combined_df.to_csv(filename, index=True, index_label='index')
        else:
            # Caso seja o primeiro salvamento
            df.to_csv(filename, index=True, index_label='index')

        print(f"Dados salvos no arquivo '{filename}' com sucesso!")

    except Exception as e:
        print(f"Erro ao salvar dados: {e}")

In [ ]:
df = pd.read_csv('db/concat_FatoFake.csv')
df.head()

,index,titulo,resumo,URL
0,0,É #FAKE vídeo de Paolla Oliveira anunciando de...,é fake post que usa vídeo de paolla oliveira p...,https://g1.globo.com/fato-ou-fake/video/e-fake...
1,1,É #FAKE que vídeo mostre criminosos deixando d...,polícia civil da bahia onde foi feita a gravaç...,https://g1.globo.com/fato-ou-fake/noticia/2025...
2,2,É #FATO: vídeo viral mostra treinamento milita...,publicada em fevereiro de na conta oficial de ...,https://g1.globo.com/fato-ou-fake/noticia/2025...
3,3,É #FAKE que mamografia aumenta o risco de cânc...,integrante da sociedade brasileira de mastolog...,https://g1.globo.com/fato-ou-fake/noticia/2025...
4,4,É #FAKE vídeo em que famosos usam camiseta com...,autor do registro é criador de conteúdos que u...,https://g1.globo.com/fato-ou-fake/noticia/2025...


In [283]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2918 entries, 0 to 2917
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   2918 non-null   int64 
 1   titulo  2918 non-null   object
 2   resumo  2916 non-null   object
 3   URL     2918 non-null   object
dtypes: int64(1), object(3)
memory usage: 91.3+ KB


In [284]:
# df = df.dropna(subset=['resumo'])
df['resumo'] = df['resumo'].fillna('não há dados sobre')


In [285]:
# Definir uma função para verificar se as frases específicas estão no 'resumo'
def remove_noticias_irrelevantes_pro_contexto(titulo):
    return ("estreia quadro no \'Mais Você\'" not in titulo) and ('apresenta o quadro \"Fato ou Fake\"' not in titulo)


In [286]:
# Aplicar a função para filtrar as linhas
df = df[df['titulo'].apply(remove_noticias_irrelevantes_pro_contexto)]

In [287]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2916 entries, 0 to 2917
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   2916 non-null   int64 
 1   titulo  2916 non-null   object
 2   resumo  2916 non-null   object
 3   URL     2916 non-null   object
dtypes: int64(1), object(3)
memory usage: 113.9+ KB


In [288]:
keys = {
    'fake': ['#FAKE', '#NÃO É BEM ASSIM']
    }

In [289]:
# Função para classificar
def classificar(row):
    # Verificar nas colunas 'titulo' e 'resumo'
    texto_completo = row['titulo'] + " " + row['resumo']  # Junta as duas colunas em uma string
    # Verifica se alguma palavra-chave de 'fake' aparece
    if any(palavra in texto_completo for palavra in keys['fake']):
        return 'fake'
    return 'fato'  # Se não encontrar 'fake', classifica como 'fato'


In [290]:
df['check'] = df.apply(classificar, axis=1)
df.head()


,index,titulo,resumo,URL,check
0,0,É #FAKE vídeo de Paolla Oliveira anunciando de...,é fake post que usa vídeo de paolla oliveira p...,https://g1.globo.com/fato-ou-fake/video/e-fake...,fake
1,1,É #FAKE que vídeo mostre criminosos deixando d...,polícia civil da bahia onde foi feita a gravaç...,https://g1.globo.com/fato-ou-fake/noticia/2025...,fake
2,2,É #FATO: vídeo viral mostra treinamento milita...,publicada em fevereiro de na conta oficial de ...,https://g1.globo.com/fato-ou-fake/noticia/2025...,fato
3,3,É #FAKE que mamografia aumenta o risco de cânc...,integrante da sociedade brasileira de mastolog...,https://g1.globo.com/fato-ou-fake/noticia/2025...,fake
4,4,É #FAKE vídeo em que famosos usam camiseta com...,autor do registro é criador de conteúdos que u...,https://g1.globo.com/fato-ou-fake/noticia/2025...,fake


In [291]:
df['check'].value_counts(normalize=True)*100

check
fake    94.890261
fato     5.109739
Name: proportion, dtype: float64

In [ ]:
print(df['titulo'][df['check'] == 'fato'])

Series([], Name: titulo, dtype: object)


In [293]:
print(df['resumo'][df['check'] == 'fato'])


2       publicada em fevereiro de na conta oficial de ...
40      após ser empossado nesta segunda ele repetiu v...
47      jovem diz que contraiu hpv na mão por equipame...
153     grupo era contratado por políticos em períodos...
179     equipe do fato ou fake apurou e concluiu com d...
                              ...                        
2877    vídeo que tem bombado nas redes sociais mostra...
2885    em são paulo candidatos apresentaram propostas...
2890    vídeos exibidos na tv globo e globonews com co...
2904    texto compartilhado no whatsapp lista providên...
2907    texto tem sido muito compartilhado na web espe...
Name: resumo, Length: 149, dtype: object


In [ ]:
saveData(df)